## Initializations

**first we Should install hazm library for cleaning and tokenizing persian text**

In [1]:
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 36.3 MB 40.6 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hazm: filename=hazm-0.7.1-py3-none-any.whl size=480662 sha256=ec208945cfd67d2642d22322fb2e5a95bd10ec8337d204ce03ffd466b15a5c8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-2rie108o/wheels/ec/21/4a/ced3ec820057abc8ec94927e5fab1848a577b55977eee1713f
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436397 sha256=26de2bfc073c563f6cc2f6dfad15a3f3ae1162103c4358bb0e2b56849327a758
  Stored in directory: /root/.cache/pip/wheels/e3/8d/39/e3aa34880de366c8b3e4584c4f5f3777289005a4b1553a9c4c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp3

**Then we import necessary libraries**

In [2]:
import re
import random
# Tokenizing Sentences
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace
from nltk.lm import MLE
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

from collections import defaultdict
import os
import io #codecs
import nltk
from itertools import product
import math
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [67]:
from hazm import Normalizer
from collections import defaultdict
import hazm

### Part A :

In this step we are going to do some Cleaning and Normalizing and then segmening the sentences.
For the normalization Part we're going to use Hazm library to:
> We remove all non persian chars.

> We correct all "ي" and replace then with "ی"

For the Segmentation part, We're going to add begginners<'s'> and enders<"e"> to sentences and then tokenize each sentence.




In [90]:
def preprocess_tokenize_Persian_hp(input_txt):
    # first step we preprocess using Hazm Library
    input_txt = Normalizer().normalize(input_txt)

    ## Second step is tokenization of sentences
    sentences = hazm.sent_tokenize(input_txt)
    tokenized_sentences = []
    # Tokenize sentences into words
    for sent in sentences:
        # clean up sentences
        sent = re.sub(r'([.؟!])', r' \1', sent)
        # substitube type of start
        sent = re.sub(r'\s+', ' ', sent)
        ## and also add start/end to the sentences
        sent = '<s> ' + sent.strip() + ' <e>'
        tokens = hazm.word_tokenize(sent)
        tokenized_sentences.append(tokens)

    return tokenized_sentences

In [91]:
hp_fa=open("data/hp_fa.txt").read()
token_hp_fa = preprocess_tokenize_Persian_hp(hp_fa)
token_hp_fa

[['<s>',
  '\ufeffآقا',
  'و',
  'خانم',
  'دورسلی',
  'ساکن',
  'خانه',
  'شماره',
  'چهار',
  'خیابان',
  'پریوت',
  'درایو',
  'بودند',
  '.',
  '<e>'],
 ['<s>',
  'خانواده',
  'آنها',
  'بسیار',
  'معمولی',
  'و',
  'عادی',
  'بود',
  'و',
  'آن\u200cها',
  'از',
  'این',
  'بابت',
  'بسیار',
  'راضی',
  'و',
  'خوشنود',
  'بودند',
  '.',
  '<e>'],
 ['<s>',
  'این',
  'خانواده',
  'به',
  'هیچ',
  'وجه',
  'با',
  'امور',
  'مرموز',
  'و',
  'اسرار',
  'آمیز',
  'سروکار',
  'نداشتند',
  'زیرا',
  'سحر',
  'و',
  'جادو',
  'را',
  'امر',
  'مهمل',
  'و',
  'بیهوده\u200cای',
  'می\u200cپنداشتند',
  'و',
  'علاقه\u200cای',
  'به',
  'این\u200cگونه',
  'مسائل',
  'نداشت',
  'ن',
  'آقای',
  'دورسلی',
  'مدیر',
  'شرکت',
  'دریل',
  'سازی',
  'گرونینگز',
  '،',
  'مردی',
  'درشت\u200cاندام',
  'و',
  'قوی',
  'هیکل',
  'بود',
  'با',
  'گردنی',
  'بسیار',
  'کوتاه',
  'که',
  'سبیل',
  'بلندی',
  'نیز',
  'داشت',
  '.',
  '<e>'],
 ['<s>',
  'همسر',
  'او',
  '،',
  'خانم',
  'دورسلی',
 

## Part B and Part C

in part B we're going to create bigram models of the tokenized sentences we created, and for that part we made train_model function that inputs tokenized_sentences and output the probabilities as it's wanted in the question.

In [65]:
def train_model(token_sent):
    # First we count each bigram
    bigram_counts = defaultdict(lambda: defaultdict(int))
    for sent in token_sent:
        sentence_length = len(sent) - 1
        for i in range(sentence_length):
            bigram_counts[sent[i]][sent[i+1]] = 1 + bigram_counts[sent[i]][sent[i+1]]
    
    # at this stage we calculate bigram probs
    # and use add-one smoothing for it
    vocabulary_size = len(bigram_counts.keys())
    bigram_probs = defaultdict(lambda: defaultdict(float))

    for word1 in bigram_counts:
        total_count = sum(bigram_counts[word1].values())
        for word2 in bigram_counts[word1]:
            bigram_probs[word1][word2] = (bigram_counts[word1][word2] + 1) / (total_count + vocabulary_size)
    return bigram_probs

In [114]:
bigram_model_persian = train_model(token_hp_fa)
# We calculated each probability for bigrams
print("probs for and example word is:")
bigram_model_persian['او']['گرفت']

probs for and example word is:


0.00031117104034851154

## Part D: Generating new sentences

Here we're going to generate 5 new sentences for the model we trained, and length of the sentences should be between 12 to 24

In [78]:
def generate_bigram_sentences(model, number_of_sentences):
    generated_sentences = []
    for i in range(number_of_sentences):
        print("sentence",i+1, " Generated:")
        # We're going to generate a sentence between 12 to 24 length and the length should be random
        while True:
          #length of each sentence should be between 12 to 24
          sentence_length = random.randint(12, 24)
          # and we start each sentence with <s>
          sentence = ['<s>']
          while len(sentence) < 24:
              prev_word = sentence[-1]
              next_word_probs = model[prev_word]
              cum_probs = []
              total_prob = 0
              for word, prob in next_word_probs.items():
                  total_prob += prob
                  cum_probs.append((word, total_prob))
              sample = random.random()
              for word, prob in cum_probs:
                  if sample <= prob:
                      sentence.append(word)
                      break
              if sentence[-1] == '<e>':
                  break
          sentence_text = ' '.join(sentence[1:-1])
          if sentence_length <= len(sentence_text):
            break
        generated_sentences.append(sentence_text)
        print(sentence_text,"\n")
    # Here we're going to make a string
    # and then return the string
    generated_sentences = '\n'.join(generated_sentences)
    return generated_sentences

In [113]:
# all of the created texts are saved in the "generated_bi_text"
generated_bi_text = generate_bigram_sentences(bigram_model_persian,5)

sentence 1  Generated:
بعد هیچ شکی هم پیدا کردم ، دوست ندارد . 

sentence 2  Generated:
بدعنق به هم رفتند که این جا می‌گرفت … . 

sentence 3  Generated:
با پشت یقه چین‌دار با تردید از قرار ملاقاتش با خانواده تونو می‌گم . 

sentence 4  Generated:
در صورت داشته_باشم که می‌لرزید تنها کاری داشتین ولی قیافه به داخل کوزه‌ی کوچکی بر رویشان نخورد . 

sentence 5  Generated:
او را روی پیشانی هری که بیست سانتیمتره . 



## **Part E** : Doing Part-B for n==3 and n==5

Here we're going to do all of the partB and Part D for n==3 and also n==5.

all first we're going to make trigrams:

**for n==3**

In [104]:
def train_3gram_model(token_sent):
    # all first we're going to count all trigrams
    cnt = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    for sent1 in token_sent:
        sent1 = ['<s>'] + sent1
        nength_sent1 = len(sent1) - 2
        for i in range(nength_sent1):
            cnt[sent1[i]][sent1[i+1]][sent1[i+2]] = cnt[sent1[i]][sent1[i+1]][sent1[i+2]] + 1
    # and second step we're going to calculate probabilities
    probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    for word1 in cnt:
        for word2 in cnt[word1]:
            total_count = sum(cnt[word1][word2].values())
            for word3 in cnt[word1][word2]:
                #trigram probs for each word is:
                probs[word1][word2][word3] = (cnt[word1][word2][word3] + 1) / (total_count + len(cnt.keys()))
    return probs

In [112]:
def generate_3gram_sentences(trigram_model, num_sentences):
    generated_sents = []
    for i in range(num_sentences):
        # in a loop we're going to generate 5 sentences:
        print("sentence",i+1 , "generated: ")
        #While we have not reached end of sentences
        while True:
          # as it's said in the question length should be between 12 to 24
          sentence_length = random.randint(12, 24)
          sentence = ['<s>', '<s>']
          #as long as we have not reached 24 tokens
          while len(sentence) < 24:
              prev_word1, prev_word2 = sentence[-2], sentence[-1]
              next_word_probs = trigram_model[prev_word1][prev_word2]
              cum_probs = []
              total_prob = 0
              for word, prob in next_word_probs.items():
                  total_prob += prob
                  cum_probs.append((word, total_prob))
              sample = random.random()
              for word, prob in cum_probs:
                  if sample <= prob:
                      sentence.append(word)
                      break
              if sentence[-1] == '<e>':
                  # break on end of sents
                  break
          sentence_text = ' '.join(sentence[2:-1])
          if sentence_length <= len(sentence_text):
            break
        generated_sents.append(sentence_text)
        print(sentence_text,"\n")
    generated_sents = '\n'.join(generated_sents)
    return generated_sents

first we train the model and get 3gram probabilities from gram counts. and then we create sentences and All of the sentences created will be saved in "generate_3gram_text"

In [105]:
trigram_model = train_3gram_model(token_hp_fa)
generate_3gram_text = generate_3gram_sentences(trigram_model,5)

sentence 1 generated: 
سانتور جواب نداد . 

sentence 2 generated: 
وضع رون به فکر داره … خب ، گفتم بابات از این گنده‌تر نمی‌شه . 

sentence 3 generated: 
بچه‌ها نباید این کارو نم کنم . 

sentence 4 generated: 
من نمی‌دونم شما از کجا می‌دونست که کسی بتواند آن‌قدر فضول باشد . 

sentence 5 generated: 
رون زیر لب به غذابزند . 



**For n==5**

In [116]:
def train_fivegram_model(tokenized_sentences):
    # first we get the count for each 5gram
    fivegram_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int)))))
    for sentence in tokenized_sentences:
        sentence = ['<s>','<s>','<s>'] + sentence
        for i in range(len(sentence) - 4):
            fivegram_counts[sentence[i]][sentence[i+1]][sentence[i+2]][sentence[i+3]][sentence[i+4]] += 1
    # then we calculate the probabiity for each 5gram
    fivegram_probabilities = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float)))))
    for word1 in fivegram_counts:
        for word2 in fivegram_counts[word1]:
            for word3 in fivegram_counts[word1][word2]:
                for word4 in fivegram_counts[word1][word2][word3]:
                    total_count = sum(fivegram_counts[word1][word2][word3][word4].values())
                    for word5 in fivegram_counts[word1][word2][word3][word4]:
                        fivegram_probabilities[word1][word2][word3][word4][word5] = (fivegram_counts[word1][word2][word3][word4][word5] + 1) / (total_count + len(fivegram_counts.keys()))
    return fivegram_probabilities

In [117]:
def generate_5gram_sentences(fivegram_model, num_sentences):
    generated_sents = []
    for i in range(num_sentences):
        print("sentence",i+1,"Generated :")
        # while we have not reached end of the sentence
        while True:
          # as it's said in the question the number of tokens should be between 12 to 24
          sentence_length = random.randint(12, 24)
          sentence = ['<s>', '<s>', '<s>', '<s>']
          # and while its les than 24 tokens
          while len(sentence) < 24:
              prev_word1, prev_word2, prev_word3, prev_word4 = sentence[-4], sentence[-3], sentence[-2],sentence[-1]
              next_word_probs = fivegram_model[prev_word1][prev_word2][prev_word3][prev_word4]
              cum_probs = []
              total_prob = 0
              for word, prob in next_word_probs.items():
                  total_prob += prob
                  cum_probs.append((word, total_prob))
              sample = random.random()
              for word, prob in cum_probs:
                  if sample <= prob:
                      sentence.append(word)
                      break
              if sentence[-1] == '<e>':
                  break
          sentence_text = ' '.join(sentence[4:-1])

          if sentence_length <= len(sentence_text):
            break
        generated_sents.append(sentence_text)
        print(sentence_text,"\n")
    generated_sents = '\n'.join(generated_sents)
    return generated_sents

In [118]:
fivegram_model = train_fivegram_model(token_hp_fa)
generated_five_text = generate_5gram_sentences(fivegram_model,5)


sentence 1 Generated :
جاروی هرمیون گرنجر فقط روی زمین غلتید و جاروی نویل اصلا تکان نخورد . 

sentence 2 Generated :
آن‌ها هرشب به انجام تکالیفی که روزبه روز روی‌هم انباشه می‌شد می‌پرداختند . 

sentence 3 Generated :
ولی حالت چهره‌ی اسنیپ وقتی چشم هری به پای او افتاد به این سادگی از یادش نمی‌رفت . 

sentence 4 Generated :
او یکی از انگشت‌های دراز و باریکش را آهسته به در کشید . 

sentence 5 Generated :
صدای تلق و تولوق چنان بلند بود که ممکن بود تمام قلعه را از خواب بیدار کرده_باشد . 



## **Part F** : Comparing Results

The five-gram model has better results than other two models. as 5-gram model has more previous words and pays more attention to previous words, so it's more like the real sentences of harryPotter.

for example the sentence created from 5 gram mode is :

> او یکی از انگشت‌های دراز و باریکش را آهسته به در کشید


and the sentence created form 3gram is :

> سانتور جواب نداد . 


so when we compare the sentences of each models, 5-gram model has the most realestic sentences and as we can see from the example generated sentence it's very close to the book.


## **Part G** : Which sentence's perplexity is more?

The Sentence "نماز ستون دین است" would be a more complicated sentence. because words: "دین" and "نماز" are not present in the dictionary of harryPotter_Fa. so the probability of occurance of these words is close to zero, and as a result the complexity of "نماز ستون دین است" is very larger than the first sentence:'هری به هاگوارتز بازگشت.' that looks like a sentence from the book.

and this is because, model is based on the sentences of the book and the probability of previous words. so as it's said earlier, probability of occurance of sentence_2 words is close to zero and the perplexity of it is larger.

In [119]:
sentence_1  = 'هری به هاگوارتز بازگشت.'
sentence_2 = 'نماز ستون دین است.'